# Python API Tutorial: Getting Started with APIs
A majority of this work is not my own! I am condensing [this API tutorial](https://www.dataquest.io/blog/python-api-tutorial/), to provide a quick and easy way for you to learn what an API is and use one to gather data for your final project!!

## What is an API?
An API, or Application Programming Interface, is a server that you can use to retrieve and send data to using code. APIs are most commonly used to retrieve data, and that will be the focus of this beginner tutorial.

When we want to receive data from an API, we need to make a `request`. Requests are used all over the web. For instance, when you visited this blog post, your web browser made a request to the Dataquest web server, which responded with the content of this web page. 

![API](api-request.svg)

API requests work in exactly the same way – you make a request to an API server for data, and it responds to your request.

## Making API Requests in Python
In order to work with APIs in Python, we need tools that will make those requests. In Python, the most common library for making requests and working with APIs is the requests library. The requests library isn’t part of the standard Python library, so you’ll need to install it to get started.

Using the Anaconda Prompt, you can install requests using the following command:
```
conda install requests
```
Once you’ve installed the library, you’ll need to import it. Let’s start with that important step:

In [2]:
import requests

Now that we’ve installed and imported the requests library, let’s start using it.

## Making Our First API Request
There are many different types of requests. The most commonly used one, a **GET** request, is used to retrieve data. Because we’ll just be working with retrieving data, our focus will be on making 'get' requests.

When we make a request, the response from the API comes with a response code which tells us whether our request was successful. Response codes are important because they immediately tell us if something went wrong.

To make a 'GET' request, we’ll use the `requests.get()` function, which requires one argument — the URL we want to make the request to. We'll start by making a request to an API endpoint that doesn't exist, so we can see what that response code looks like.

In [3]:
response = requests.get("http://api.open-notify.org/this-api-doesnt-exist")

The `get()` function returns a `response` object. We can use the `response.status_code` attribute to receive the status code for our request:

In [4]:
response.status_code

404

The '404' status code might be familiar to you — it's the status code that a server returns if it can’t find the file we requested. In this case, we asked for `this-api-doesnt-exist` which (surprise, surprise) didn't exist!

Let's learn a little more about common status codes.

## API Status Codes
Status codes are returned with every request that is made to a web server. Status codes indicate information about what happened with a request. Here are some codes that are relevant to GET requests:

- `200`: Everything went okay, and the result has been returned (if any).
- `301`: The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
- `400`: The server thinks you made a bad request. This can happen when you don’t send along the right data, among other things.
- `401`: The server thinks you’re not authenticated. Many APIs require login credentials, so this happens when you don’t send the right credentials to access an API.
- `403`: The resource you’re trying to access is forbidden: you don’t have the right permissions to see it.
- `404`: The resource you tried to access wasn’t found on the server.
- `503`: The server is not ready to handle the request.

You might notice that all of the status codes that begin with a ‘4’ indicate some sort of error. The first number of status codes indicate their categorization. This is useful — you can know that if your status code starts with a ‘2’ it was successful and if it starts with a ‘4’ or ‘5’ there was an error. If you’re interested you can read more about status codes [here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

## API Documentation
In order to ensure we make a successful request, when we work with APIs it's important to consult the documentation. Documentation can seem scary at first, but as you use documentation more and more you’ll find it gets easier.

We’ll be working with the Open Notify API, which gives access to data about the international space station. It’s a great API for learning because it has a very simple design, and doesn’t require authentication. We’ll teach you how to use an API that requires authentication in a later post.

Often there will be multiple APIs available on a particular server. Each of these APIs are commonly called **endpoints**. The first endpoint we’ll use is `http://api.open-notify.org/astros.json`, which returns data about astronauts currently in space.

If you click the link above to look at the documentation for this endpoint, you’ll see that it says *This API takes no inputs*. This makes it a simple API for us to get started with. We’ll start by making a GET request to the endpoint using the requests library:

In [5]:
response = requests.get("http://api.open-notify.org/astros.json")
response.status_code

200

We received a ‘200’ code which tells us our request was successful. The documentation tells us that the API response we’ll get is in JSON format. In the next section we’ll learn about JSON, but first let’s use the `response.json()` method to see the data we received back from the API:

In [6]:
response.json()

{'number': 7,
 'message': 'success',
 'people': [{'name': 'Mark Vande Hei', 'craft': 'ISS'},
  {'name': 'Oleg Novitskiy', 'craft': 'ISS'},
  {'name': 'Pyotr Dubrov', 'craft': 'ISS'},
  {'name': 'Thomas Pesquet', 'craft': 'ISS'},
  {'name': 'Megan McArthur', 'craft': 'ISS'},
  {'name': 'Shane Kimbrough', 'craft': 'ISS'},
  {'name': 'Akihiko Hoshide', 'craft': 'ISS'}]}

## Working with JSON Data in Python with Pandas
[JSON](http://json.org/) (JavaScript Object Notation) is the language of APIs. JSON is a way to encode data structures that ensures that they are easily readable by machines. JSON is the primary format in which data is passed back and forth to APIs, and most API servers will send their responses in JSON format.

You might have noticed that the JSON output we received from the API looked like it contained Python dictionaries, lists, strings and integers. You can think of JSON as being a complex container of these objects represented as strings. Let’s use the example above to brign the JSON into a Pandas DataFrame:

In [7]:
import pandas as pd
pd.DataFrame.from_dict(data=response.json(), orient='index')

,0
number,7
message,success
people,"[{'name': 'Mark Vande Hei', 'craft': 'ISS'}, {..."


I realize this doesn't look pretty, we would want to continue manipulating this data to get all of the columns we want in our DataFrame. Each API is unique for the task at hand and, as such, will return the JSON format for that task.

## Using an API with Query Parameters
The `http://api.open-notify.org/astros.json` endpoint we used earlier does not take any parameters. We just send a GET request and the API sends back data about the number of people currently in space.

It’s very common, however, to have an API endpoint that requires us to specify parameters. An example of this the `http://api.open-notify.org/iss-pass.json` endpoint. This endpoint tells us the next times that the international space station will pass over a given location on the earth.

If we look at the documentation, it specifies required `lat` (latitude) and `lon` (longitude) parameters.

We can do this by adding an optional keyword argument, `params`, to our request. We can make a dictionary with these parameters, and then pass them into the `requests.get()` function. Here’s what our dictionary would look like, using coordinates for New York City: 

In [8]:
parameters = {
    'lat': 40.71,
    'lon': -74
}

We can also do the same thing directly by adding the parameters directly to the URL. like this: 'http://api.open-notify.org/iss-pass.json?lat=40.71&lon;=-74'.

It’s almost always preferable to setup the parameters as a dictionary, because 'requests' takes care of some things that come up, like properly formatting the query parameters, and we don’t need to worry about inserting the values into the URL string.

Let’s make a request using these coordinates and see what response we get. 

In [9]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)
response.status_code

200

In [10]:
response.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1620397639,
  'latitude': 40.71,
  'longitude': -74.0,
  'passes': 5},
 'response': [{'duration': 614, 'risetime': 1620445376},
  {'duration': 644, 'risetime': 1620451161},
  {'duration': 576, 'risetime': 1620457046},
  {'duration': 578, 'risetime': 1620462918},
  {'duration': 647, 'risetime': 1620468732}]}

In [12]:
pd.DataFrame.from_dict(data=response.json(), orient='index', dtype=object)

,0
message,success
request,"{'altitude': 100, 'datetime': 1620397639, 'lat..."
response,"[{'duration': 614, 'risetime': 1620445376}, {'..."


## Conclusion
Now you know the basics of what an API is and how to use one! For the final project, you will need to use an API to gather your data for the project. I will provide you the URL and the parameters required. Good luck!